# 🤖 Ders 2 Uygulaması: Denetimli Öğrenme ve Eğri Uydurma (Curve Fitting) Analizi

Bu proje, Ders 2 notlarında işlenen **Denetimli Öğrenme (Supervised Learning)** felsefesini kod düzeyinde modelleyerek, algoritmaların veriden "kural çıkarma" (f(x) fonksiyonunu bulma) sürecini detaylandırmaktadır.

### Sistem Bileşenleri ve Amaç
Denetimli öğrenme modeline iki temel bileşen sunulur:
1.  **Öznitelik (Feature / Input - X)**: Tahmin yapmak için kullanılacak veridir (Örn: Evin metrekaresi).
2.  **Etiket (Label / Output - Y)**: Makinenin ulaşması beklenen doğru sonuçtur (Örn: Evin gerçek fiyatı).

Amacımız, makinenin X ile Y arasındaki görünmez ilişkiyi matematiksel bir eğriye ($f(x)$) dönüştürmesi (Curve fitting) ve bunu yaparken gelecekteki verileri de doğru tahmin edebilmesidir.

### 3 Temel Öğrenme Senaryosunun Matematiksel İspatı
Kod çalıştırıldığında, aynı veri seti üzerinde eğitilen 3 farklı modelin "Eğitim" (ders çalışma) ve "Test" (gerçek hayat) performansları konsola yazdırılır ve grafiğe dökülür:

* **Öğrenememe (Underfitting)**: Algoritma çok basittir. Ev büyüdükçe fiyatın arttığı yönündeki o bariz ana eğilimi (trendi) hiç anlayamaz. Düz ve anlamsız bir çizgi çeker.Hem eğitimde hem testte başarısızdır.
* **Genelleme (Generalization)**: İdeal ve ulaşılması hedeflenen durumdur. Makine, verilerdeki ufak tefek sapmalara (gürültüye) aldırış etmeden işin "mantığını" kavramıştır. Eğitim hatası makul bir seviyededir; ancak asıl başarısı, daha önce hiç görmediği "Test" evlerinin fiyatını yüksek isabetle bilmesidir.
* **Ezberleme (Overfitting)**: En tehlikeli makine öğrenmesi hatasıdır. Model ana kuralı bulmak yerine eğitim setindeki tüm evlerin fiyatlarını noktası noktasına ezberler. Eğitim hatası neredeyse sıfırdır. Ancak hiç görmediği yeni bir metrekare sorulduğunda, çizdiği o kusursuz (!) kural çöker ve tamamen mantıksız tahminler üretir.

*Not: Bu eğri uydurma mantığı, dizilimleri tahmin eden GPT gibi devasa Transformer tabanlı dil modellerinin de temelini oluşturur.*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

print("-" * 70)
print("DERS 2: DENETİMLİ ÖĞRENME VE EĞRİ UYDURMA (CURVE FITTING) ANALİZİ")
print("-" * 70)

# --- 1. ADIM: VERİ SETİ (ÖZNİTELİK VE ETİKET) OLUŞTURMA ---
# Öznitelik (Feature / Input - X): Evin metrekaresi.
# Etiket (Label / Output - Y): Evin gerçek piyasa fiyatı.
np.random.seed(42)

# Eğitim için 30 adet ev verisi (Sistemin ders çalışacağı veriler)
X_egitim_ham = np.sort(np.random.rand(30) * 150 + 50) 
Y_egitim_kural = np.cos(X_egitim_ham / 25) * 1000 + (X_egitim_ham * 20) 
Y_egitim_gercek = Y_egitim_kural + np.random.randn(30) * 300 # Fiyatlara gerçek dünya gürültüsü ekliyoruz

# Sınav için 100 adet yepyeni ev verisi (Sistemin hiç görmediği veriler)
X_test_ham = np.linspace(50, 200, 100)
Y_test_gercek = np.cos(X_test_ham / 25) * 1000 + (X_test_ham * 20)

X_egitim = X_egitim_ham[:, np.newaxis]
y_egitim = Y_egitim_gercek
X_test = X_test_ham[:, np.newaxis]
y_test = Y_test_gercek

print("BİLGİ: Sistem 30 adet ev verisi (X, Y çifti) ile eğitilip, hiç görmediği 100 ev ile test edilecektir.\n")

# --- 2. ADIM: MAKİNE ÖĞRENMESİ MODELLERİNİN KURULUMU ---
# Sistem, f(x) kuralını bulmak için 3 farklı karmaşıklık seviyesinde çalıştırılacaktır.
modeller = {
    "1. ÖĞRENEMEME (Underfitting)": make_pipeline(PolynomialFeatures(1), LinearRegression()),
    "2. GENELLEME (Generalization)": make_pipeline(PolynomialFeatures(4), LinearRegression()),
    "3. EZBERLEME (Overfitting)": make_pipeline(PolynomialFeatures(20), LinearRegression())
}

# --- 3. ADIM: EĞİTİM, TEST VE GÖRSELLEŞTİRME ---
fig = plt.figure(figsize=(16, 6))
fig.canvas.manager.set_window_title("Makine Öğrenmesi: Öğrenme Senaryoları")

for i, (model_adi, model) in enumerate(modeller.items()):
    ax = plt.subplot(1, 3, i + 1)
    
    # 1. Aşama: Model elindeki 30 evle ders çalışıyor (Fit)
    model.fit(X_egitim, y_egitim)
    
    # 2. Aşama: Kendi çalıştığı sorulardan deneme sınavı (Eğitim Hatası)
    tahmin_egitim = model.predict(X_egitim)
    hata_egitim = mean_squared_error(y_egitim, tahmin_egitim)
    
    # 3. Aşama: Gerçek Sınav! Hiç görmediği 100 ev soruluyor (Test Hatası)
    tahmin_test = model.predict(X_test)
    hata_test = mean_squared_error(y_test, tahmin_test)

    # Konsol Raporlaması
    print(f"[{model_adi}]")
    print(f"   -> Eğitim Hatası (Ders çalışma performansı) : {int(hata_egitim)}")
    print(f"   -> Test Hatası (Gerçek hayat performansı)   : {int(hata_test)}")
    if "EZBERLEME" in model_adi:
        print("   -> ANALİZ: Dikkat! Eğitim hatası çok düşük ama test hatası devasa. Model veriyi ezberlemiş!\n")
    elif "GENELLEME" in model_adi:
        print("   -> ANALİZ: Harika! Hem eğitim hem test hatası dengeli ve düşük. Ana trend kavranmış.\n")
    else:
        print("   -> ANALİZ: Model kapasitesi yetersiz, iki testte de başarısız. Trendi anlayamamış.\n")

    # Grafik Çizimleri
    plt.scatter(X_egitim_ham, y_egitim, color='blue', s=40, edgecolor='black', label='Eğitim Verisi (Evler)')
    plt.plot(X_test_ham, tahmin_test, color='red', linewidth=2.5, label='Modelin Bulduğu Kural (f(x))')
    
    plt.title(f"{model_adi}", fontsize=11, fontweight='bold', pad=15)
    plt.xlabel("Ev Büyüklüğü (Öznitelik - X)")
    plt.ylabel("Ev Fiyatı (Etiket - Y)")
    
    # Görsel Yorumlar
    plt.text(55, y_egitim.max() + 400, f"Ders Hatası: {int(hata_egitim)}\nSınav Hatası: {int(hata_test)}", 
             fontsize=10, bbox=dict(facecolor='white', alpha=0.8, edgecolor='gray'))
    
    plt.ylim(y_egitim.min() - 500, y_egitim.max() + 1000)
    plt.legend(loc='lower right', fontsize=9)
    plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## Veri sayısını 30'dan 300'e çıkartırsak ne olur ?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

print("-" * 70)
print("DERS 2: BÜYÜK VERİ (BIG DATA) İLE EĞRİ UYDURMA ANALİZİ")
print("-" * 70)

# --- 1. ADIM: BÜYÜK VERİ SETİ OLUŞTURMA ---
np.random.seed(42)

# Veri sayısını 30'dan 300'e çıkardık!
veri_sayisi = 300

X_egitim_ham = np.sort(np.random.rand(veri_sayisi) * 150 + 50) 
Y_egitim_kural = np.cos(X_egitim_ham / 25) * 1000 + (X_egitim_ham * 20) 
Y_egitim_gercek = Y_egitim_kural + np.random.randn(veri_sayisi) * 300 # Gürültü eklenmiş piyasa fiyatları

# Sınav için 150 adet yepyeni ev verisi
X_test_ham = np.linspace(50, 200, 150)
Y_test_gercek = np.cos(X_test_ham / 25) * 1000 + (X_test_ham * 20)

X_egitim = X_egitim_ham[:, np.newaxis]
y_egitim = Y_egitim_gercek
X_test = X_test_ham[:, np.newaxis]
y_test = Y_test_gercek

print(f"BİLGİ: Sistem {veri_sayisi} adet ev verisi (X, Y çifti) ile eğitilip, hiç görmediği 150 ev ile test edilecektir.\n")

# --- 2. ADIM: MAKİNE ÖĞRENMESİ MODELLERİNİN KURULUMU ---
modeller = {
    "1. ÖĞRENEMEME (Underfitting)": make_pipeline(PolynomialFeatures(1), LinearRegression()),
    "2. GENELLEME (Generalization)": make_pipeline(PolynomialFeatures(4), LinearRegression()),
    "3. EZBERLEME (Overfitting)": make_pipeline(PolynomialFeatures(20), LinearRegression())
}

# --- 3. ADIM: EĞİTİM, TEST VE GÖRSELLEŞTİRME ---
fig = plt.figure(figsize=(16, 6))
fig.canvas.manager.set_window_title(f"Makine Öğrenmesi: Büyük Veri ({veri_sayisi} Örnek)")

for i, (model_adi, model) in enumerate(modeller.items()):
    ax = plt.subplot(1, 3, i + 1)
    
    # Modelin eğitimi
    model.fit(X_egitim, y_egitim)
    
    # Eğitim Hatası (Ders Çalışma)
    tahmin_egitim = model.predict(X_egitim)
    hata_egitim = mean_squared_error(y_egitim, tahmin_egitim)
    
    # Test Hatası (Gerçek Hayat Sınavı)
    tahmin_test = model.predict(X_test)
    hata_test = mean_squared_error(y_test, tahmin_test)

    # Konsol Raporlaması
    print(f"[{model_adi}]")
    print(f"   -> Eğitim Hatası: {int(hata_egitim)}")
    print(f"   -> Test Hatası  : {int(hata_test)}")
    if "EZBERLEME" in model_adi:
        print("   -> ANALİZ: Artan veri sayısı ezberci modeli zorladı. Zikzaklar azaldı ama test hatası hala yüksek.\n")
    elif "GENELLEME" in model_adi:
        print("   -> ANALİZ: Harika! Model bol veriyle ana kuralı kusursuz kavradı.\n")
    else:
        print("   -> ANALİZ: Model kapasitesi yetersiz, veri artsa da düz çizgi çekmekten kurtulamadı.\n")

    # Grafik Çizimleri
    # Veri sayısı çok olduğu için noktaların boyutunu (s=15) küçültüp, şeffaflık (alpha=0.5) ekledik
    plt.scatter(X_egitim_ham, y_egitim, color='blue', s=15, alpha=0.5, edgecolor='none', label='Eğitim Verisi (300 Ev)')
    plt.plot(X_test_ham, tahmin_test, color='red', linewidth=3, label='Modelin Eğrisi (f(x))')
    
    plt.title(f"{model_adi}", fontsize=11, fontweight='bold', pad=15)
    plt.xlabel("Ev Büyüklüğü (Öznitelik - X)")
    plt.ylabel("Ev Fiyatı (Etiket - Y)")
    
    plt.text(55, y_egitim.max() + 200, f"Ders Hatası: {int(hata_egitim)}\nSınav Hatası: {int(hata_test)}", 
             fontsize=10, bbox=dict(facecolor='white', alpha=0.8, edgecolor='gray'))
    
    plt.ylim(y_egitim.min() - 500, y_egitim.max() + 1000)
    plt.legend(loc='lower right', fontsize=9)
    plt.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# 📈 Analiz: Veri Sayısını 30'dan 300'e Çıkarmak Neleri Değiştirdi?

Makine öğrenmesinin en ünlü kurallarından biri şudur: **"Daha fazla veri, her zaman daha iyi ve daha akıllı bir model demektir."** Ders 2 uygulamamızda ev sayısını 30'dan 300'e çıkardığımızda (Big Data simülasyonu), üç farklı öğrenme senaryomuzda şu dramatik değişiklikler gözlemlenmiştir:

### 1. Öğrenememe (Underfitting) Üzerindeki Etkisi
* **Sonuç:** Hiçbir şey değişmedi.
* **Neden?** Model kapasitesi o kadar düşüktür ki (sadece düz bir çizgi çizebilir), ona dünyadaki bütün evlerin verisini de verseniz o evler arasındaki karmaşık ve kıvrımlı ilişkiyi anlayamaz. Kapasitesiz bir modele fazla veri vermek onu akıllandırmaz.

### 2. Genelleme (Generalization) Üzerindeki Etkisi
* **Sonuç:** Model çok daha sarsılmaz ve güvenilir hale geldi.
* **Neden?** 30 ev varken model ana kuralı zaten bulmuştu. Ancak veri 300'e çıktığında, aradaki istisnai (çok ucuz veya çok pahalı) evlerin yarattığı gürültü (noise) etkisini yitirdi. Model, kalabalığın bilgeliğinden yararlanarak asıl fiyat artış trendini ($f(x)$) kusursuz bir şekilde kilitledi. Test hatası daha da düştü.

### 3. Ezberleme (Overfitting) Üzerindeki Etkisi (En Büyük Değişim!)
* **Sonuç:** Çılgın zikzaklar azaldı, model "uslandı" ama hala hatalı.
* **Neden?** 30 ev varken 20. dereceden (çok karmaşık) bir model, o 30 noktanın hepsine değebilmek için aralarda devasa, mantıksız zikzaklar çizebiliyordu. Ancak sahaya 300 ev döküldüğünde veri o kadar sıklaştı ki, algoritma boşluk bulup o saçma sapan zikzakları yapamadı. 
* **Çıkarım:** Veri sayısını artırmak, Ezberleme (Overfitting) problemini çözmenin en doğal ve en güçlü yoludur. Veri arttıkça, model noktaları ezberlemekte zorlanır ve ister istemez genel kuralı öğrenmeye doğru itilir.